In [1]:
import json
from collections import Counter, defaultdict
from wiki_labels import qid_lab_get, qid_en_desc_get
from wiki_location import q2cc
from text_utils import cl
import sys
sys.path.insert(0, '/projekti/mondoAPI')
from pnu.parse import parse
from api.db import db
import pandas as pd
import gzip

sys.path.insert(0, '/projekti/nelma')
sys.path.insert(0, '/projekti/nelma/mondoDB')

from mondoDB.parse_thai_name import norm_thai, tokenise, parse_thai

from tqdm.notebook import tqdm

print(qid_lab_get(42, 'en').keys(), parse('davor lauc')['tags'][0], parse('반기문')['tags'][0])

loading from airtable
saving
dict_keys(['douglas adams']) {'tokens': ['davor', 'lauc'], 'labels': ('fn1_hr_Latn_HR', 'ln_hr_Latn_HR'), 'prob': 92.06} {'tokens': ['반', '기문'], 'labels': ('ln_ko_Hang_KR', 'fn1_ko_Hang_KR'), 'prob': 97.09}


In [9]:
# wiki lang to iso
WIKI_DIR = '/backup/wikidata'
api_key = open('airtable_key.txt').read()
from pyairtable import Table, Api
import sys
sys.path.insert(0, '/projekti/nelma')
from mondoDB.referencedb import provi

if True:
    lang2cc = defaultdict(Counter)
    for k, v in provi.items():
        lang, s, c = v['id'].split('_')
        bod = 0#float(v['Percent'])*3 if 'Percent' in v else 0
        if 'Status' in v:
            if v['Status'] == 'official':
                bod += 5
            elif v['Status'] == 'official_regional':
                bod += 2
            elif v['Status'] == 'de_facto_official':
                bod += 2
            elif v['Status'] == 'romanized':
                bod += 0.0
            elif v['Status'] == 'foreign':
                bod += -0.5
        if bod>1.1:
            lang2cc[f'{lang}'][c] = max(bod, lang2cc[f'{lang}'][c])
    api = Api(api_key)
    w2iso = {t['fields']['WMF']: t['fields']['qid'] if 'qid' in t['fields'] else None for t in Table(api_key, 'appUZvAm9EHZgC1Eg', 'wiki_lang').all()}
    wiki2cc = defaultdict(Counter)
    for k, v in w2iso.items():
        if len(k) == 2:
            wiki2cc[v].update(lang2cc[k])
        elif k.count('-') == 1:
            a, b = k.split('-')
            if len(a) == 2 and len(b) == 2:
                wiki2cc[v][b.upper()] += 5 
            elif len(a) == 2 and len(b) == 4:
                #wiki2cc[v].update(lang2cc[f'{k}_{b.uppser()}'])
                wiki2cc[v].update(lang2cc[a])
    wikil2cc = {k: {cc: round(100*v2/max(v.values())) for cc, v2 in v.items()} for k, v in wiki2cc.items() if len(v)>0 and max(v.values())>0}

cc2lang = defaultdict(set)
for lang, ccs in lang2cc.items():
    for cc in ccs:
        if ccs[cc]>1:
            cc2lang[cc].add(lang)
iso2w = {k:v for v, k in w2iso.items()}

wikil2cc['Q1860']['US'], cc2lang['CH'], cc2lang['US']

(40, {'de', 'fr', 'it', 'rm'}, {'en', 'es'})

In [47]:
from wikilang2iso import get_wiki_cc, cc_weights

for i, l in tqdm(enumerate(gzip.open('/backup/wikidata/wikinelma.jsonl.gz')), total=17_630_406):
    j = json.loads(l)
    if j['type'] == 'per':
        qid = int(j['wiki_id'][1:])
        cc, ccs = get_wiki_cc({k: v for k, v in j in k in cc_weights})
        break 
ccs

  0%|          | 0/17630406 [00:00<?, ?it/s]

[('US', 15.4)]

In [ ]:
{'a': 1}+{'b': 2}

In [15]:
notable = pd.read_csv('/backup/wikidata/cross-verified-database.csv.gz', compression='gzip', encoding='latin-1')
pd.set_option('display.max_columns', None)
qrank = {v['wikidata_code']:v['ranking_visib_5criteria'] for _, v in notable.iterrows()}
notable

,wikidata_code,birth,death,updated_death_date,approx_birth,approx_death,birth_min,birth_max,death_min,death_max,gender,level1_main_occ,name,un_subregion,birth_estimation,death_estimation,bigperiod_birth_graph_b,bigperiod_death_graph_b,curid,level2_main_occ,freq_main_occ,freq_second_occ,level2_second_occ,level3_main_occ,bigperiod_birth,bigperiod_death,wiki_readers_2015_2018,non_missing_score,total_count_words_b,number_wiki_editions,total_noccur_links_b,sum_visib_ln_5criteria,ranking_visib_5criteria,all_geography_groups,string_citizenship_raw_d,citizenship_1_b,citizenship_2_b,list_areas_of_rattach,area1_of_rattachment,area2_of_rattachment,list_wikipedia_editions,un_region,group_wikipedia_editions,bplo1,dplo1,bpla1,dpla1,pantheon_1,level3_all_occ
0,Q1000002,1932.0,1990.0,NaN,NaN,NaN,1932.0,1932.0,1990.0,1990.0,Male,Culture,Claus_Hammel,Western Europe,1932.0,1990.0000,5.Contemporary period 1901-2020AD,5.Contemporary period 1901-2020AD,2949539,Culture-core,0.800000,0.200000,Culture-periphery,playwright,5.Contemporary period 1901-2020AD,5.Contemporary period 1901-2020AD,1669,3,1777,1,11,18.083672,1058542.0,Germany,'Germany',Germany,NaN,D:_'Germany'_matchB1_P:_'Germany',Germany,Missing,dewiki,Europe,grB,11.833333,12.420000,53.416668,54.381390,0,D:_playwright_journalist_writer_screenwriter_P...
1,Q1000005,1860.0,1927.0,NaN,NaN,NaN,1860.0,1860.0,1927.0,1927.0,Male,Culture,Karel_MatÄj_Äapek-Chod,Western Europe,1860.0,1927.0000,4.Mid Modern Period 1751-1900AD,5.Contemporary period 1901-2020AD,4217319,Culture-core,0.538462,0.307692,Culture-periphery,writer,4.Mid Modern Period 1751-1900AD,5.Contemporary period 1901-2020AD,25008,3,6491,9,15,23.980610,131428.0,Czech_Republic,'Czech_Republic',Czech_Republic,NaN,D:_'Czech_Republic'_mismatchB2_P:_'Czech_Repub...,Old_(before_year_1993_AD)_Czech_Republic,Missing,dewiki|cswiki|enwiki|eowiki|itwiki|kkwiki|rowi...,Europe,grA,12.929798,14.421389,49.440605,50.087502,0,D:_writer_journalist_P:_naturalist_writer_jour...
2,Q1000006,1971.0,NaN,NaN,NaN,NaN,1971.0,1971.0,NaN,NaN,Male,Culture,Florian_Eichinger,Western Europe,1971.0,2053.8447,5.Contemporary period 1901-2020AD,5.Contemporary period 1901-2020AD,5050967,Culture-core,1.000000,NaN,Missing,film,5.Contemporary period 1901-2020AD,Missing,27285,3,1573,1,10,20.666656,775768.0,Germany,'Germany',Germany,NaN,D:_'Germany'_matchB1_P:_'Germany',Germany,Missing,dewiki,Europe,grB,9.191944,NaN,48.897499,NaN,0,D:_film_screenwriter_film_P:_regisseur_autor_f...
3,Q1000015,1983.0,NaN,NaN,NaN,NaN,1983.0,1983.0,NaN,NaN,Male,Culture,Florian_Jahr,Western Europe,1983.0,2067.1899,5.Contemporary period 1901-2020AD,5.Contemporary period 1901-2020AD,2588583,Culture-core,1.000000,NaN,Missing,actor,5.Contemporary period 1901-2020AD,Missing,37331,3,1931,1,10,21.185040,691735.0,Germany,'Germany',Germany,NaN,D:_'Germany'_matchB1_P:_'Germany',Germany,Missing,dewiki,Europe,grB,13.383333,NaN,52.516666,NaN,0,D:_actor_P:_schauspiel_German
4,Q1000023,1912.0,1977.0,NaN,NaN,NaN,1912.0,1912.0,1977.0,1977.0,Female,Leadership,Wiltraut_Rupp-von_BrÃ¼nneck,Western Europe,1912.0,1977.0000,5.Contemporary period 1901-2020AD,5.Contemporary period 1901-2020AD,922120,Administration/Law,0.833333,0.166667,Politics,judge,5.Contemporary period 1901-2020AD,5.Contemporary period 1901-2020AD,2955,3,1578,1,6,17.996210,1103282.0,Germany,'Germany',Germany,NaN,D:_'Germany'_matchB1_P:_'Germany',Old_(before_year_1990_AD)_Germany,Missing,dewiki,Europe,grB,13.350000,8.400000,52.433300,49.016666,0,D:_judge_jurist_P:_ richter_verfassung_German
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2291812,Q999994,1988.0,NaN,NaN,NaN,NaN,1988.0,1988.0,NaN,NaN,Male,Sports/Games,Mitja_MeÅ¾nar,Southern Europe,1988.0,2065.3259,5.Contemporary period 1901-2020AD,5.Contemporary period 1901-2020AD,20581328,Sports/Games,0.900000,NaN,Missing,ski,5.Contemporary period 1901-2020AD,Miss

In [37]:
fo = gzip.open('/backup/wikidata/wiki_person.tsv.gz', 'wt')
fo.write('qid\tname\tfn\tln\tgender\tcc_first\tnative_lang\tccs\tdesc_en\tdesc_nl\tdob\tdod\timage\tsort\n')
for i, l in tqdm(enumerate(gzip.open('/backup/wikidata/wikinelma.jsonl.gz')), total=25_042_218):
    j = json.loads(l)
    if j['type'] == 'per':
        qid = int(j['wiki_id'][1:])
        cc, ccs = get_wiki_cc(j['country']+j['nationality'], j['residence']+j['birth_place']+j['death_place']+j['positions']+j['educated_at']+j['works_at'], j['native_language'])
        name = None
        if j['native_language']:
            langs = Counter([iso2w[q[5:]] for q in j['native_language'] if q[5:] in iso2w])
        else:
            langs = Counter()
        langs.update(cc2lang[cc])
        native_lang = langs.most_common()[0][0] if langs else ''
        if '-' in native_lang:
            native_lang = native_lang.split('-')[0]
        elif len(native_lang)>2:
            native_lang = native_lang[:2]
        if j['name_native']:
            name = cl(j['name_native'][0]).lower()
        else:
            for l, f in langs.most_common():
                name = qid_lab_get(qid, l)
                if name:
                    name = list(name)[0]
                    break
        if not name:
            try:
                name = list(qid_lab_get(qid, 'en', include_alt=False).keys())[0]
            except:
                try:
                    name = list(j['l'].values())[0][0]
                except:
                    name = ''
        if name:
            namel = name.lower().strip()
            desc = qid_en_desc_get(qid)
            dob = j['dob'][0][:4] if j['dob'] else ''
            dod = j['dod'][0][:4] if 'dod' in j and j['dod'] else ''
            
            image = j['picture'][0] if j['picture'] else ''
            sort = qrank[j['wiki_id']] if  j['wiki_id'] in qrank  else (100_000_000+(int(dob) if dob else 1980)-(50 if image else 0)-(10 if desc else 0))

            fn = ''
            if j['fn']:
                fns = qid_lab_get(int(j['fn'][0][6:]), include_alt=True).keys()
                fns = Counter({a:len(a) for a in fns if a in namel})
                if len(fns)>0:
                    fn = fns.most_common()[0][0]

            ln = ''
            if j['ln']:
                fns = qid_lab_get(int(j['ln'][0][6:]), include_alt=True).keys()
                fns = Counter({a:len(a) for a in fns if a in namel})
                if len(fns)>0:
                    ln = fns.most_common()[0][0]
            # only 
            gender = ''
            if j['gender']==['WIKI_Q6581097']:
                gender = 'm'
            elif j['gender']==['WIKI_Q6581072']:
                gender = 'f'
            
            if False and not fn or not ln:
                if ',' in namel:
                    if ' ' in namel.split(',')[0].strip():
                        namel = namel.split(',')[0].strip()
                    else:
                        namel = namel.split(',')[1].strip()+' '+namel.split(',')[0].strip()
                    #print(name, namel)
                parts = namel.split(' ')
                if len(parts) == 1 and fn:
                    if namel.startswith(fn):
                        parts = [fn, namel[len(fn):]]
                    elif namel.endswith(fn):
                        parts = [fn, namel[:len(namel)-len(fn)]]
                if len(parts) == 1 and ln:
                    if namel.startswith(ln):
                        parts = [namel[len(ln):], ln]
                    elif namel.endswith(ln):
                        parts = [namel[:len(namel)-len(ln)], ln]

                if len(parts) == 2:
                    if fn and parts[0]==fn:
                        ln = parts[1]
                    elif fn and parts[1]==fn:
                        ln = parts[0]
                    elif ln and parts[1]==ln:
                        fn = parts[0]
                    elif ln and parts[0]==ln:
                        fn = parts[1]
                    else:
                        fn, ln = parts
                elif len(parts) > 2:
                    fn, ln = parts[0], parts[-1]
                    
            if not cc:
                cc = ''

            desc_en = qid_en_desc_get(qid)
            for q in j['positions']:
                desc_nl = list(qid_lab_get(int(q[6:]), native_lang, include_alt=False).keys())
                desc_nl = desc_nl[0] if desc_nl else ''

                if desc_nl:
                    break


            fo.write(f'{qid}\t{name}\t{fn}\t{ln}\t{gender}\t{cc}\t{native_lang}\t{ccs}\t{desc_en}\t{desc_nl}\t{dob}\t{dod}\t{image}\t{sort}\n')
    #if i>100:
    #    break
fo.close()
    

  0%|          | 0/25042218 [00:00<?, ?it/s]

In [51]:
#t = pd.read_csv('/backup/wikidata/wiki_person.tsv.gz', sep='\t', on_bad_lines='warn',)
t[t['cc_first']=='TH'].to_csv('/projekti/thainame/data/notable/wiki.csv.gz')


mkdir: cannot create directory ‘/projekti/thainame/data’: File exists


In [ ]:
fo = open('/backup/wikidata/wiki_parsed.tsv', 'w')
fi = open('/backup/wikidata/wiki_person.tsv', 'r')
fi.readline()
fo.write(f"qid\tname\tprob\ttitle\tfn\tln\tfn_org\tln_org\n") 

for i, l in tqdm(enumerate(fi), total=9928121 ):
    qid, name, fn_org, ln_org, gender, desc, cc, dob, image, sort = l.strip('\n').split('\t')
    rec = parse(name)
    titles, fn, ln = [], [], []
    prob = -1 
    if rec['tags'] and rec['tags'][0]['prob']>0.2:
        prob = rec['tags'][0]['prob']
        for tok, typ in zip(rec['tags'][0]['tokens'], rec['tags'][0]['labels']):
            if typ[:5] == 'title':
                titles.append(tok)
            elif typ[:2] == 'fn':
                fn.append(tok)
            elif typ[:2] == 'ln':
                ln.append(tok)
    fo.write(f"{qid}\t{name}\t{prob}\t{','.join(titles)}\t{','.join(fn)}\t{','.join(ln)}\t{fn_org}\t{ln_org}\n") 
fo.close()

In [ ]:
!wc -l /backup/wikidata/wiki_person.tsv

In [ ]:
uk = 0
fn = '/projekti/mondonomo/books/thai/thai_wiki.csv'
fo = open(fn, 'w')
fo2 = open(fn.replace('thai_', 'nonthai_'), 'w')
for i, l in enumerate(open('/backup/wikidata/wiki_person.jsonl')):
    j = json.loads(l)
    cc = get_wiki_cc(j['country'], j['birth_place'], j['language'])
    if 'en' in j['l'] and 'th' in j['l'] and cc:
        native = j["name_native"][0] if j["name_native"] else ''
        en = j["l"]["en"][0].replace(',',' ')
        th = j["l"]["th"][0].replace(',',' ')
        if cc == 'TH':
            fo.write(f'{j["id"]},{cc},{en},{th}\n')
        else:
            fo2.write(f'{j["id"]},{cc},{en},{th}\n')
        uk += 1
fo.close()
!head /projekti/mondonomo/books/thai/thai_wiki.csv

In [ ]:
from tqdm.notebook import tqdm
uk = 0
fn = '/backup/wikidata/hr_wiki.csv'
fo = open(fn, 'w')
fo2 = open(fn.replace('hr_', 'nonhr_'), 'w')
for l in tqdm(open('/backup/wikidata/wiki_person.jsonl'), total=10_044_571):
    j = json.loads(l)
    cc = get_wiki_cc(j['country'], j['birth_place'], j['language'])
    if 'en' in j['l'] and 'hr' in j['l'] and cc:
        native = j["name_native"][0] if j["name_native"] else ''
        en = j["l"]["en"][0].replace(',',' ')
        th = j["l"]["hr"][0].replace(',',' ')
        if cc == 'HR':
            fo.write(f'{j["id"]},{cc},{en},{th}\n')
        else:
            fo2.write(f'{j["id"]},{cc},{en},{th}\n')
        uk += 1
fo.close()
!head $fn

In [ ]:
parse('George Washington')